<a href="https://colab.research.google.com/github/campospluiza/procesamientoDelHabla/blob/main/Campos_CASO_DE_ESTUDIO_clasificaci%C3%B3n_de_textos_usando_redes_neuronales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DESAFIO OBLIGATORIO - Clasificación de textos usando redes neuronales

# CONSIGNAS QUE DEBE RESPONDER

## 1) Describa las principales características del dataset utilizado.

El dataset utilizado fue el "IMDB movie review sentiment classification
", un dataset que es un compilado de opiniones publicadas en IMDB, la más popular fuente de contenido sobre películas y TV. Las opiniones, en este caso, era sobre películas. El dataset 50.000 reviews (25.000 de entreanamiento y 25.000 de prueba), 50.000 opiniones sin etiqueta para realizar modelos de aprendizaje no supervisado

## 2) Deben comentar su experiencia al intentar correr el mismo código en sus computadoras: cuáles errores surgieron, cómo los solucionaron, sugerencias que brindan, si debieron o no modificar algo del código para solucionar algún error o poder correr el ejemplo en sus máquinas.

En mi caso fue necesário instalar la librerí keras, que no la tenía disponible. Pero tardó un poco en correr. Elegí trabajar directamente en colab.

## 3) Explique con sus palabras el pre-procesamiento que realiza en el dataset para poder utilizarlo para entrenar la red neuronal

El preprocesamiento del dataset es una etapa clave para que la red neuronal pueda trabajar con los datos de manera correcta. Primero, se limpia el texto eliminando todo lo que no es necesario, como etiquetas HTML y signos de puntuación, y se convierte todo a minúsculas para mantener un formato uniforme. Luego, se usa un proceso de tokenización y vectorización, que convierte las palabras en números, asignando un índice específico a cada una para formar un vocabulario. Esto permite que la red trabaje con datos numéricos en lugar de texto, facilitando su análisis.

También se hace una división del dataset, generalmente un 80% para el entrenamiento y un 20% para validación, asegurando que el modelo pueda aprender y luego ser evaluado. Para la organización de los datos, se los agrupa en lotes, lo que ayuda a que el entrenamiento sea más eficiente. Por último, se utiliza una herramienta como `TextVectorization` para transformar todo el texto en secuencias numéricas que la red neuronal pueda entender y procesar correctamente. Todo esto asegura que los datos estén listos y optimizados para ser utilizados en el entrenamiento del modelo.

## Postee también el resultado final de evaluación del modelo ( la salida del apartado "Evaluate the model on the test set)
782/782 ━━━━━━━━━━━━━━━━━━━━ 46s 58ms/step - accuracy: 0.8349 - loss: 0.5194
[0.5190317034721375, 0.8361600041389465]


## 4) En este ejemplo para evaluar el modelo utiliza la métrica accuracy. ¿Qué opina al respecto? ¿Está bien/mal? ¿sumaría otra métrica?

Usar solo la accuracy puede ser insuficiente si el dataset tiene desbalance. Incluir métricas como el F1-score o analizar la AUC-ROC ayudaría a tener una evaluación más completa y precisa del modelo.


## 5) Comente alguna parte que le haya llamado la atención y por qué.

Lo que más me llama la atención de la clasificación de textos es cómo las bibliotecas y herramientas como Keras hacen posible que una tarea tan compleja se pueda implementar de forma relativamente accesible. Es impresionante ver cómo estas herramientas permiten convertir texto en algo que una red neuronal puede procesar y aprender, algo que al principio parece casi mágico. Aunque trabajar con Keras y otras librerías se me hace MUY difícil, siempre es asombroso ver los resultados que se pueden obtener, y cómo se puede enseñar a una máquina a entender y clasificar texto de manera efectiva. La forma en que se combinan todas las etapas, desde la limpieza y preparación del texto hasta el entrenamiento del modelo, es algo que me parece increíble.

## 6) Escriba las dudas que hayan quedado de realizar este trabajo.

Trabajar en la clasificacion de textos siempre me parece muy difícil. Más que nada me quedan dudas sobre como funcionan las librerias, todo que se puede hacer, como ser lo mas preciso posible... Me hace ver que necesito mucho estudio.

In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
import tensorflow as tf
import numpy as np
from keras import layers

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  21.0M      0  0:00:03  0:00:03 --:--:-- 21.0M


In [3]:
!ls aclImdb
!ls aclImdb/test
!ls aclImdb/train

imdbEr.txt  imdb.vocab	README	test  train
labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt
labeledBow.feat  neg  pos  unsup  unsupBow.feat  urls_neg.txt  urls_pos.txt  urls_unsup.txt


In [4]:
!cat aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

In [5]:
!rm -r aclImdb/train/unsup

In [6]:
batch_size = 32
raw_train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


In [7]:
# It's important to take a look at your raw data to ensure your normalization
# and tokenization will work as expected. We can do that by taking a few
# examples from the training set and looking at them.
# This is one of the places where eager execution shines:
# we can just evaluate these tensors using .numpy()
# instead of needing to evaluate them in a Session/Graph context.
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

# Preparar Data

In [8]:
import string
import re


# Having looked at our data above, we see that the raw text contains HTML break
# tags of the form '<br />'. These tags will not be removed by the default
# standardizer (which doesn't strip HTML). Because of this, we will need to
# create a custom standardization function.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )


# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# Now that the vectorize_layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

# Let's make a text-only dataset (no labels):
text_ds = raw_train_ds.map(lambda x, y: x)
# Let's call `adapt`:
vectorize_layer.adapt(text_ds)

# Vectorizar Opcion 1

In [9]:
text_input = keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input)
x = layers.Embedding(max_features + 1, embedding_dim)(x)
...

Ellipsis

# Vectorizar Opcion 2

In [10]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)


# Construir Modelo

In [11]:
# A integer input for vocab indices.
inputs = keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Treinar Modelo

In [12]:
epochs = 3

# Fit the model using the train and test datasets.
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 138s 216ms/step - accuracy: 0.5979 - loss: 0.6213 - val_accuracy: 0.8684 - val_loss: 0.3101
Epoch 2/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 137s 209ms/step - accuracy: 0.8857 - loss: 0.2835 - val_accuracy: 0.8720 - val_loss: 0.3055
Epoch 3/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 210ms/step - accuracy: 0.9447 - loss: 0.1504 - val_accuracy: 0.8476 - val_loss: 0.4919


# Evalular el Modelo

In [13]:
model.evaluate(test_ds)

782/782 ━━━━━━━━━━━━━━━━━━━━ 46s 58ms/step - accuracy: 0.8349 - loss: 0.5194


[0.5190317034721375, 0.8361600041389465]

# Hacer modelo end-to-end

In [14]:
# A string input
inputs = keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

# Our end to end model
end_to_end_model = keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

782/782 ━━━━━━━━━━━━━━━━━━━━ 38s 48ms/step - accuracy: 0.8352 - loss: 0.0000e+00


[0.0, 0.0, 0.8361600041389465, 0.8361600041389465]